<a href="https://colab.research.google.com/github/aharrisonau/Task-2.2C/blob/main/Task_2_2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task 2.2C**

This file is based on Practical 2, modified to complete task 2.2C.



# **Environment Import and Setup**

We use a maze as the environment under consideration and proceed to install the required system dependencies


In [3]:
# install required system dependencies
!apt-get install -y xvfb x11-utils  
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* \
            gym[box2d]==0.17.* 
!pip install pyglet
!pip install ffmpeg
! pip install pyvirtualdisplay
!pip install Image
!pip install gym-maze-trustycoder83

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 993 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 993 kB in 1s (792 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1.

If the directory vid exists and has videos left over from previous tries, its better to clean it up before continuing.

In [1]:
!mkdir ./vid
!rm ./vid/*.*

rm: cannot remove './vid/*.*': No such file or directory


We now proceed to initialise the monitor wrapper for Gym so we can visualise the maze and the agent on a video

In [1]:
import sys
# import pygame
import numpy as np
# import math
# import base64
# import io
# import IPython
import gym
import gym_maze

# from gym.wrappers import Monitor
# from IPython import display
from pyvirtualdisplay import Display
from gym.wrappers.monitoring import video_recorder

d = Display()
d.start()

# Recording filename
video_name = "./vid/Practical_2.mp4"

# Setup the environment for the maze
env = gym.make("maze-sample-10x10-v0")

##### Disabled all video to speed up execution ######
# Setup the video
#vid = None
#vid = video_recorder.VideoRecorder(env,video_name)

# env = gym.wrappers.Monitor(env,'./vid',force=True)
current_state = env.reset()



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


The code below has been updated to implement task 2.2C

Modified sections are indicated.

As an initial check of the gridsearch and for comparison, the original model from practical 2 was run

In [4]:
# original model run with the grid search.
# as gamma was not used in the original model, it will be used to 
# vary the learning rate
states_dic = {} #dictionary to keep the states/coordinates of the Q table
count = 0
for i in range(10):
    for j in range(10):
        states_dic[i, j] = count
        count+=1
        
n_actions = env.action_space.n


# Number of episode we will run
n_episodes = 10

# Maximum of iteration per episode
max_iter_episode = 500

######## Task 2.2 modifications START #########
# set a range of epsilons values to try for the e-greedy algorithm
epsilons = [0.01,0.05,0.1,0.5]
# epsilons = [0.1] # single value for testing

# set a range of gamma values to use
gammas = [0.01,0.05,0.1,0.5,1.0]
# gammas = [0.1] # single value for testing

# loop through the combinations of epsilon and gamma and collect
# the reward curves for each
rewards_per_combination = {} 
for i in range(len(epsilons)):
    for j in range(len(gammas)):

        ######## Task 2.2 modifications START #########
        # Initialize the Q-table to a realistic initiation.  For almost all actions
        # the reward will be -0.1 / 100 = -0.1e-3 for the 10x10 maze
        # so set the initial Q_table to be this
        Q_table = np.zeros((len(states_dic),n_actions))
        ######## Task 2.2 modifications END #########

        epsilon = epsilons[i]
        gamma = gammas[j]
        lr = gamma # learning rate for original Practical 2
        print()
        print('epsilon = ',epsilon,' gamma = ', gamma)
        
        # set an empty list to collect results
        rewards_per_episode = list()
######## Task 2.2 modifications END #########

        # Iterate over episodes
        for e in range(n_episodes):
          print(e, end=" ") # get a heartbeat to know we are moving
    
          # We are not done yet
          done = False
    
          # Sum the rewards that the agent gets from the environment
          total_episode_reward = 0

          for n in range(max_iter_episode): 

            #env.unwrapped.render()
            #vid.capture_frame()
            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

######## Task 2.2 modifications START #########
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample() # with probability epsilon, explore
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates])) # otherwise exploit
######## Task 2.2 modifications END #########

            next_state, reward, done, _ = env.step(action)

            next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
            next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

######## Task 2.2 modifications START #########
            # Update our Q-table using the Q-learning iteration
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            alpha = 1 / (n+1) # loop index starts at 0, so add 1
            Q_table[current_Q_table_coordinates, action] = (1-lr)*Q_table[current_Q_table_coordinates, action] + lr*(reward + max(Q_table[next_Q_table_coordinates,:]))         
######## Task 2.2 modifications END #########

            total_episode_reward = total_episode_reward + reward
            # If the episode is finished, we leave the for loop
            if done:
              break
            current_state = next_state
######## Task 2.2 modifications START #########    
          #Reset enviroment for next episode
          current_state = env.reset()
    
          rewards_per_episode.append(total_episode_reward)

        # save the rewards list for this epsilon-gamma combination
        rewards_per_combination[i,j]=rewards_per_episode
        # end of the episode loop (on e)
    #end of the gammas loop (on j)
# end of the epsilons loop (on i)
######## Task 2.2 modifications END #########
        # Save video episode and close
#print("Video successfuly saved.")
#vid.close()
#vid.enabled = False


epsilon =  0.01  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  1.0
0 

In [5]:
import pandas as pd
import statistics

av_total_rewards = pd.DataFrame(columns=gammas)

for i in range(len(epsilons)):
    for j in range(len(gammas)):
      av_total_rewards.loc[epsilons[i],gammas[j]]=statistics.mean(rewards_per_combination[i,j])




In [6]:
# print out the average rewards for the original Practical 2
pd.options.display.float_format = '{:.3f}'.format
av_total_rewards

,0.010,0.050,0.100,0.500,1.000
0.010,-0.286,0.238,0.223,0.660,0.740
0.050,-0.288,0.154,0.335,0.565,0.606
0.100,-0.138,0.166,0.343,0.572,0.579
0.500,-0.399,0.071,0.273,0.538,0.165


In [2]:
states_dic = {} #dictionary to keep the states/coordinates of the Q table
count = 0
for i in range(10):
    for j in range(10):
        states_dic[i, j] = count
        count+=1
        
n_actions = env.action_space.n


# Number of episode we will run
n_episodes = 10

# Maximum of iteration per episode
max_iter_episode = 500

######## Task 2.2 modifications START #########
# set a range of epsilons values to try for the e-greedy algorithm
epsilons = [0.01,0.05,0.1,0.5]
# epsilons = [0.1] # single value for testing

# set a range of gamma values to use
gammas = [0.01,0.05,0.1,0.5,1.0]
# gammas = [0.1] # single value for testing

# loop through the combinations of epsilon and gamma and collect
# the reward curves for each
rewards_per_combination = {} 
for i in range(len(epsilons)):
    for j in range(len(gammas)):

        ######## Task 2.2 modifications START #########
        # Initialize the Q-table to a realistic initiation.  For almost all actions
        # the reward will be -0.1 / 100 = -0.1e-3 for the 10x10 maze
        # so set the initial Q_table to be this
        Q_table = np.full((len(states_dic),n_actions),-0.1e-3)
        ######## Task 2.2 modifications END #########

        epsilon = epsilons[i]
        gamma = gammas[j]
        print()
        print('epsilon = ',epsilon,' gamma = ', gamma)
        
        # set an empty list to collect results
        rewards_per_episode = list()
######## Task 2.2 modifications END #########

        # Iterate over episodes
        for e in range(n_episodes):
          print(e, end=" ") # get a heartbeat to know we are moving
    
          # We are not done yet
          done = False
    
          # Sum the rewards that the agent gets from the environment
          total_episode_reward = 0

          for n in range(max_iter_episode): 

            #env.unwrapped.render()
            #vid.capture_frame()
            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

######## Task 2.2 modifications START #########
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample() # with probability epsilon, explore
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates])) # otherwise exploit
######## Task 2.2 modifications END #########

            next_state, reward, done, _ = env.step(action)

            next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
            next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

######## Task 2.2 modifications START #########
            # Update our Q-table using the Q-learning iteration
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            alpha = 1 / (n+1) # loop index starts at 0, so add 1
            Q_table[current_Q_table_coordinates, action] = (Q_table[current_Q_table_coordinates, action] 
                                                        + alpha*(reward - gamma * Q_table[current_Q_table_coordinates, action]))
            # this implements the required Q update strategy
            # Q(n+1) = Q(n) + alpha * (R - gamma * Q(n))
######## Task 2.2 modifications END #########

            total_episode_reward = total_episode_reward + reward
            # If the episode is finished, we leave the for loop
            if done:
              break
            current_state = next_state
######## Task 2.2 modifications START #########    
          #Reset enviroment for next episode
          current_state = env.reset()
    
          rewards_per_episode.append(total_episode_reward)

        # save the rewards list for this epsilon-gamma combination
        rewards_per_combination[i,j]=rewards_per_episode
        # end of the episode loop (on e)
    #end of the gammas loop (on j)
# end of the epsilons loop (on i)
######## Task 2.2 modifications END #########
        # Save video episode and close
#print("Video successfuly saved.")
#vid.close()
#vid.enabled = False


epsilon =  0.01  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  1.0
0 

In [9]:
import pandas as pd
import statistics

av_total_rewards = pd.DataFrame(columns=gammas)

for i in range(len(epsilons)):
    for j in range(len(gammas)):
      av_total_rewards.loc[epsilons[i],gammas[j]]=statistics.mean(rewards_per_combination[i,j])




In [10]:
# print out the average rewards for the realistic initialization
pd.options.display.float_format = '{:.3f}'.format
av_total_rewards

,0.010,0.050,0.100,0.500,1.000
0.010,-0.389,-0.267,-0.500,-0.377,-0.383
0.050,-0.294,-0.389,-0.385,-0.271,-0.398
0.100,-0.381,-0.281,-0.277,-0.388,-0.500
0.500,-0.391,-0.276,-0.373,-0.392,-0.382


Repeat the process with a optimistic initialization.

The maximum reward is 1, so optimize with this value, expecting to achieve the maximum reward on every step, which is very optomistic.

In [35]:
states_dic = {} #dictionary to keep the states/coordinates of the Q table
count = 0
for i in range(10):
    for j in range(10):
        states_dic[i, j] = count
        count+=1
        
n_actions = env.action_space.n

# Number of episode we will run
n_episodes = 10

# Maximum of iteration per episode
max_iter_episode = 500

######## Task 2.2 modifications START #########
# set a range of epsilons values to try for the e-greedy algorithm
epsilons = [0.01,0.05,0.1,0.5]
# epsilons = [0.1] # single value for testing

# set a range of gamma values to use
gammas = [0.01,0.05,0.1,0.5,1.0]
# gammas = [0.1] # single value for testing

# loop through the combinations of epsilon and gamma and collect
# the reward curves for each
rewards_per_combination = {} 
for i in range(len(epsilons)):
    for j in range(len(gammas)):
        # Initialize the Q-table to an optomistic initiation.
        # The maximum reward will be 1
        # so set the initial Q_table to be this
        # for each combination
        Q_table = np.full((len(states_dic),n_actions),1.0)
        

        epsilon = epsilons[i]
        gamma = gammas[j]
        print()
        print('epsilon = ',epsilon,' gamma = ', gamma)
        
        # set an empty list to collect results
        rewards_per_episode = list()
######## Task 2.2 modifications END #########

        # Iterate over episodes
        for e in range(n_episodes):
          print(e, end=" ") # get a heartbeat to know we are moving
    
          # We are not done yet
          done = False
    
          # Sum the rewards that the agent gets from the environment
          total_episode_reward = 0

          for n in range(max_iter_episode): 

            #env.unwrapped.render()
            #vid.capture_frame()
            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

######## Task 2.2 modifications START #########
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample() # with probability epsilon, explore
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates])) # otherwise exploit
######## Task 2.2 modifications END #########

            next_state, reward, done, _ = env.step(action)

            next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
            next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

######## Task 2.2 modifications START #########
            # Update our Q-table using the Q-learning iteration
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            alpha = 1 / (n+1) # loop index starts at 0, so add 1
            Q_table[current_Q_table_coordinates, action] = (Q_table[current_Q_table_coordinates, action] 
                                                        + alpha*(reward - gamma * Q_table[current_Q_table_coordinates, action]))
            # this implements the required Q update strategy
            # Q(n+1) = Q(n) + alpha * (R - gamma * Q(n))
######## Task 2.2 modifications END #########

            total_episode_reward = total_episode_reward + reward
            # If the episode is finished, we leave the for loop
            if done:
              break
            current_state = next_state
######## Task 2.2 modifications START #########    
          #Reset enviroment for next episode
          current_state = env.reset()
    
          rewards_per_episode.append(total_episode_reward)

        # save the rewards list for this epsilon-gamma combination
        rewards_per_combination[i,j]=rewards_per_episode
        # end of the episode loop (on e)
    #end of the gammas loop (on j)
# end of the epsilons loop (on i)
######## Task 2.2 modifications END #########
        # Save video episode and close
#print("Video successfuly saved.")
#vid.close()
#vid.enabled = False


epsilon =  0.01  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.01  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.05  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.1  gamma =  1.0
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.01
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.05
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.1
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  0.5
0 1 2 3 4 5 6 7 8 9 
epsilon =  0.5  gamma =  1.0
0 

In [36]:
import pandas as pd
import statistics

av_total_rewards = pd.DataFrame(columns=gammas)

for i in range(len(epsilons)):
    for j in range(len(gammas)):
      av_total_rewards.loc[epsilons[i],gammas[j]]=statistics.mean(rewards_per_combination[i,j])




In [37]:
# print out the average rewards for the optomistic initialization
pd.options.display.float_format = '{:.3f}'.format
av_total_rewards

,0.010,0.050,0.100,0.500,1.000
0.010,-0.388,-0.385,-0.291,-0.387,-0.394
0.050,-0.287,-0.371,-0.390,-0.392,-0.500
0.100,-0.270,-0.388,-0.378,-0.292,-0.394
0.500,-0.277,-0.277,-0.281,-0.500,-0.283
